#### *Instalando as bibliotecas que serão necessárias*

In [1]:
# Instala a biblioteca "pandas" que será usada para manipulação de dados.
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Instala a biblioteca "numpy" que será usada para operações numéricas.
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Instala a biblioteca "yfinance" que será usada para obter dados financeiros.
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# Instala a biblioteca "plotly" que será usada para a exibição de gráficos. 
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable


#### *Importando as bibliotecas que serão necessárias*

In [5]:
# Importa a biblioteca "pandas", que será usada para manipulação de dados.
import pandas as pd
# Importa a biblioteca "numpy", que será usada para operações numéricas.
import numpy as np
# Importa a biblioteca "yfinance" para obter alguns dados de certas ações.
import yfinance as yf
# Importa o módulo "graph_objects" da biblioteca "plotly" para lidar com a exibição de gráficos.
import plotly.graph_objects as go
# Importa o módulo datetime da biblioteca "datetime" para lidar com datas.
from datetime import datetime, timedelta
# Importa a biblioteca "math" para lidar com certas operações matemáticas.
import math
# Importa o módulo Optional da biblioteca "typing" para lidar com a padronização dos tipos de parâmetros opcionais das funções.
from typing import Optional

#### *Definindo funções para processar os arquivos .xlsx*

In [6]:
def process_file_and_get_results(filename: str) -> pd.DataFrame:
    '''
        Description:

        Args:

        Return:
    '''

    # Ler todas as planilhas do arquivo Excel
    dfs = pd.read_excel(filename, sheet_name=None)
    
    #
    first_trading_day = dfs['Período 1'][dfs['Período 1'].columns[0]][0].split(" - ")[0]
    #
    first_trading_day = np.datetime64(first_trading_day)
    
    #
    results = pd.DataFrame({
        "Retorno": 0
    }, index=[first_trading_day])
    
    # Acessar e trabalhar com cada DataFrame
    for period, df in dfs.items():
        if not df.empty:
            #
            period_last_trading_day = df[df.columns[0]][0].split(" - ")[1]
            #
            period_last_trading_day = np.datetime64(period_last_trading_day)
            #
            tickers_to_trade = math.ceil(df.shape[0]*0.1)
            #
            period_return = df.iloc[:tickers_to_trade]['Retorno Real'].mean() # Trocar por um .sum() ?
            #
            result = pd.DataFrame({
                "Retorno": period_return,
            },index=[period_last_trading_day])
            #
            results = pd.concat([results, result])
    
    return results
            

In [7]:
def process_files_and_get_results(filesname: list) -> list:
    '''
        Description:

        Args:

        Return:
    '''
    
    #
    results = []
    
    #
    for xlsx_number, xlsx in enumerate(filesname):
        #
        result = process_file_and_get_results(xlsx)['Retorno']
        #
        result.name = f"Retornos da Estratégia na Previsão {xlsx_number}"
        #
        results.append(result)
        
    return results

In [8]:
def get_average_result(results: list) -> pd.Series:
    '''
        Description:

        Args:

        Return:

    '''
    
    # Garantir que todos os resultados em results tenham o mesmo tamanho (para poder inicializar o average_result).
    
    
    average_result = np.zeros(len(results[0]))
        
    for result in results:
        average_result += result
    
    average_result = average_result/len(results)
    
    average_result.name = "Retornos da Média das Previsões"
    
    return average_result

In [9]:
def get_ticker_standardized_returns(symbol, reference_data, strategy_time_period: str, 
                                    standardized_returns_name="", standardized_cumulative_returns_name="") -> pd.Series:
    '''
        Description:

        Args:

        Return:
    '''

    #
    period = 5 if (strategy_time_period == "W") else 20
    #
    periods_number = len(reference_data)
    
    #
    result = np.zeros(periods_number)
    
    #
    data_extraction_initial_day = reference_data.index[0]
    #
    data_extraction_final_day = reference_data.index[-1] + timedelta(1) # O time delta 1 é por que o yf não inclui o último dia de 
                                                                        # negociação passado como parâmetro. Logo, somando um timedelta(1),
                                                                        # tal dia será incluido na extração de dados do S&P500 index.
                                                                            
    #
    adj_close = yf.download(symbol, data_extraction_initial_day, data_extraction_final_day)['Adj Close']
    
    #
    result[0] = np.log((adj_close[period - 1])/adj_close[0])*100
    #
    for period_number in range(1, periods_number):
        #
        result[period_number] = np.log((adj_close[(period)*period_number - 1 + (period)])/(adj_close[(period)*period_number]))*100
        
    #
    result = pd.Series(result, reference_data.index)
    #
    cumulative_result = result.cumsum()
    #
    result.name = standardized_returns_name
    #
    cumulative_result.name = standardized_cumulative_returns_name
    
    return result, cumulative_result

#### *Definindo algumas variáveis que serão importantes para a exibição de resultados*

In [10]:
setup = {
    #
    "monthly_results_sheets": ['resultados_portfolio_mensal_full_1.xlsx', 'resultados_portfolio_mensal_full_2.xlsx',
                               'resultados_portfolio_mensal_full_3.xlsx', 'resultados_portfolio_mensal_full_4.xlsx',
                               'resultados_portfolio_mensal_full_5.xlsx'],
    #
    "weekly_results_sheets": ['resultados_portfolio_semanal_full_1.xlsx']
}

In [11]:
#
monthly_results = process_files_and_get_results(setup['monthly_results_sheets'])

In [12]:
#
weekly_results = process_file_and_get_results(setup['weekly_results_sheets'][0]) # O .xlsx semanal é processado desse jeito pois, por hora,
                                                                                 # só possuímos 1 dele.

In [13]:
#
average_monthly_results = get_average_result(monthly_results)

In [14]:
# Posso usar qualquer "result" em "results" como reference_data, já que, ambos "result" possuem as mesmas datas como índices.

sp500_index_period_returns, sp500_index_cumulative_returns = get_ticker_standardized_returns('^GSPC', monthly_results[0], "M",
                                                                                             "Retornos do S&P500 Index",
                                                                                             "Retornos Acumulados do S&P500 Index")

[*********************100%%**********************]  1 of 1 completed


#### *Exibindo e explicando as variáveis definidas na seção anterior*

In [15]:
#
monthly_results

[2019-02-22    0.000000
 2019-03-21    4.859427
 2019-04-18    4.864098
 2019-05-17    5.933391
 2019-06-17    5.581319
                 ...   
 2023-08-31    1.373854
 2023-09-29    2.502946
 2023-10-27    4.109963
 2023-11-27    6.950656
 2023-12-26    7.329663
 Name: Retornos da Estratégia na Previsão 0, Length: 61, dtype: float64,
 2019-02-22    0.000000
 2019-03-21    6.267345
 2019-04-18    4.779837
 2019-05-17    5.699338
 2019-06-17    5.581320
                 ...   
 2023-08-31    1.373857
 2023-09-29    2.502941
 2023-10-27    4.253324
 2023-11-27    6.746918
 2023-12-26    7.329658
 Name: Retornos da Estratégia na Previsão 1, Length: 61, dtype: float64,
 2019-02-22    0.000000
 2019-03-21    5.314519
 2019-04-18    4.692802
 2019-05-17    6.400990
 2019-06-17    5.637725
                 ...   
 2023-08-31    1.373850
 2023-09-29    2.502945
 2023-10-27    4.826801
 2023-11-27    6.950660
 2023-12-26    7.329661
 Name: Retornos da Estratégia na Previsão 2, Length: 61, dtype

In [16]:
#
monthly_results[0]

2019-02-22    0.000000
2019-03-21    4.859427
2019-04-18    4.864098
2019-05-17    5.933391
2019-06-17    5.581319
                ...   
2023-08-31    1.373854
2023-09-29    2.502946
2023-10-27    4.109963
2023-11-27    6.950656
2023-12-26    7.329663
Name: Retornos da Estratégia na Previsão 0, Length: 61, dtype: float64

In [17]:
#
average_monthly_results

2019-02-22    0.000000
2019-03-21    5.577003
2019-04-18    5.178876
2019-05-17    5.855372
2019-06-17    5.292026
                ...   
2023-08-31    1.364823
2023-09-29    2.502944
2023-10-27    4.364492
2023-11-27    7.130327
2023-12-26    7.537228
Name: Retornos da Média das Previsões, Length: 61, dtype: float64

In [18]:
#
weekly_results

,Retorno
2019-01-31,0.000000
2019-02-06,-0.119412
2019-02-13,-0.023905
2019-02-21,0.110191
2019-02-28,0.447097
...,...
2023-11-27,-0.243826
2023-12-04,0.677020
2023-12-11,0.474322
2023-12-18,-0.061220


In [19]:
#
monthly_results[0]

2019-02-22    0.000000
2019-03-21    4.859427
2019-04-18    4.864098
2019-05-17    5.933391
2019-06-17    5.581319
                ...   
2023-08-31    1.373854
2023-09-29    2.502946
2023-10-27    4.109963
2023-11-27    6.950656
2023-12-26    7.329663
Name: Retornos da Estratégia na Previsão 0, Length: 61, dtype: float64

In [20]:
#
sp500_index_period_returns

2019-02-22    2.203167
2019-03-21    3.657078
2019-04-18   -1.679795
2019-05-17    1.725725
2019-06-17    2.914529
                ...   
2023-08-31    0.659504
2023-09-29   -5.174368
2023-10-27   -4.069676
2023-11-27    8.806866
2023-12-26    4.714021
Name: Retornos do S&P500 Index, Length: 61, dtype: float64

In [21]:
#
sp500_index_cumulative_returns

2019-02-22     2.203167
2019-03-21     5.860246
2019-04-18     4.180451
2019-05-17     5.906176
2019-06-17     8.820705
                ...    
2023-08-31    52.594225
2023-09-29    47.419857
2023-10-27    43.350180
2023-11-27    52.157046
2023-12-26    56.871067
Name: Retornos Acumulados do S&P500 Index, Length: 61, dtype: float64

#### *Definindo algumas funções que serão úteis para plotar gráficos*

In [22]:
def plot_time_serie_line_graph(serie: pd.Series, serie_title: Optional[str] = "", serie_xaxis_title: Optional[str] = "", 
                               serie_yaxis_title: Optional[str] = "") -> None:
    '''
        Description:
            Essa função é responsável por realizar o plot de uma série temporal.
        Args:
            serie (pd.Series): Série temporal cujos dados serão utilizados para o plot. Os valores de tal série temporal estarão no eixo y,
            enquanto os índices dessa série estarão no eixo x.
            title (string):
            xaxis_title (string):
            yaxis_title (string):
            
        Return:
            Essa função plota a série temporal, mas não retorna nada.
    '''
    
    # Cria a figure onde a série temporal em questão será plotada.
    fig = go.Figure()
    
    # Adiciona o gráfico da série temporal em questão ao plot.
    fig.add_trace(go.Scatter(
        # Define os valores do eixo x do plot.
        x = serie.index,
        # Define os valores do eixo y do plot.
        y = serie.values,
        # Define o tipo do gráfico que será plotado.
        mode = "lines"
    ))
    
    # Adiciona algumas legendas ao plot.
    fig.update_layout(
        # Adiciona um título ao plot.
        title = serie_title,
        # Adiciona uma legenda ao eixo x do plot.
        xaxis_title = serie_xaxis_title,
        # Adiciona uma legenda ao eixo y do plot.
        yaxis_title = serie_yaxis_title
    )
    
    # Exibe o plot criado.
    fig.show()

In [23]:
def plot_multiples_time_series_line_graphs(data_list: list, serie_title: Optional[str] = "", serie_xaxis_title: Optional[str] = "", 
                               serie_yaxis_title: Optional[str] = "") -> None:
    '''
        Description:
            Esta função exibe em um mesmo plot múltiplos gráficos de séries temporais. 
        Args:
            data_list (list): Lista contendo as séries temporais cujos gráficos serão exibidos no plot. Os valores de tais séries temporais
                              estarão no eixo y, enquanto os índices dessas séries estarão no eixo x.
            title (string) [Optional]: Título do plot.
            xaxis_title (string) [Optional]: Título do eixo x do plot.
            yaxis_title (string) [Optional]: Título do eixo y do plot.
        Return:
            None: A função exibe os gráficos, mas não retorna nenhum valor.  
        Errors:
            TypeError: É esperado que todas os elementos da lista "data_list" sejam objetos do tipo pd.Series, isto é, que sejam séries temporais.
            ValueError: É esperado que todas as séries temporais presentes na variável "data_list" possuam os mesmos índices. 
            TypeError: É esperado que todas as séries temporais presentes na variável data_list possuam um atributo "name".
    '''    

    # Verifica se todos os elementos presentes na lista "data_list" são séries temporais.
    are_all_data_time_series = all(isinstance(df, pd.Series) for df in data_list)
    
    # Retorna um erro caso algum dos dados presentes na variável "data_list" não seja uma série temporal
    if not are_all_data_time_series:
        raise TypeError("Todos os dados presentes no parâmetro 'data_list' devem ser séries temporais.")
    
    # Verifica se as séries temporais presentes na variável "data_list" possuem os mesmos índices.
    are_all_index_equal = all(df.index.equals(data_list[0].index) for df in data_list)
    
    # Retorna um erro caso as séries temporais possuam índices diferentes.
    if not are_all_index_equal:
        raise ValueError("Todos as séries temporais devem possuir os mesmos índices.")
    
    # Verifica se todas as séries temporais presentes na variável "data_list" possuem o atributo "name".
    all_time_series_have_names = all(hasattr(df,"name") for df in data_list)
    
    # Retorna um erro caso uma das séries temporais presentes na variável "data_list" não possua o atributo "name".
    if not all_time_series_have_names:
        raise TypeError("Todas as séries temporais devem possuir o atributo 'name'")
    
    # Cria uma lista de timestamps que representará o eixo x do gráfico que será plotado.
    x_axis = data_list[0].index.tolist() # Observe que só podemos fazer isso pois temos certeza que todas as séries temporais possuem os mesmos índices.
    # Cria a figura onde será plotado o gráfico.
    fig = go.Figure()

    # Adiciona cada série temporal ao gráfico.
    for time_serie in data_list:
        # Plota o gráfico (Data x Valor da Ação) do ticker em questão.
        fig.add_trace(go.Scatter(x=x_axis, y=time_serie.values, mode="lines", name=time_serie.name))
    
    # Atualiza o layout para permitir destaque ao clicar na legenda.
    fig.update_layout(
        # Seta um título para o plot.
        title=serie_title,
        # Seta um título para o eixo x do plot.
        xaxis_title=serie_xaxis_title,
        # Seta um título para o eixo y do plot.
        yaxis_title=serie_yaxis_title,   
    )
    
    # Exibe o gráfico criado.
    fig.show()

#### *Exibindo os Resultados*

##### **Gráfico de Retornos da Média das Previsões vs. Retornos do S&P 500 Index**

In [24]:
plot_multiples_time_series_line_graphs([average_monthly_results, sp500_index_period_returns], "Retornos da Média das Previsões vs. Retornos do S&P 500 Index", "Período", "Retornos (Log.)")

##### **Gráfico de Retorno Acumulado da Estratégia vs. Retorno Acumulado do S&P 500 Index**

In [25]:
#
average_monthly_cumulative_returns = average_monthly_results.cumsum()
#
average_monthly_cumulative_returns.name = "Retornos Acumulados da Média das Previsões"
#
plot_multiples_time_series_line_graphs([average_monthly_cumulative_returns, sp500_index_cumulative_returns], "Retorno Acumulado da Estratégia vs. Retorno Acumulado do S&P 500 Index", "Período", "Retornos Acumulado")

##### **Gráfico de dispersão (Adicionar área / intervalo de confiança para a linha de tendencia)**

In [32]:
# 
fig = go.Figure(data=go.Scatter(
    #
    x=sp500_index_period_returns,
    #
    y=average_monthly_results,
    #
    mode='markers',
    name="Pontos de Dispersão"
))

# Calcula a linha de tendência (regressão linear)
slope, intercept = np.polyfit(sp500_index_period_returns, average_monthly_results, 1)  # Retorna o coeficiente angular (slope) e o intercept
line = slope * sp500_index_period_returns + intercept  # Equação da linha de tendência

# Adiciona a linha de tendência
fig.add_trace(go.Scatter(
    x= sp500_index_period_returns, 
    y=line, 
    mode='lines', 
    name='Linha de Tendência', 
    line=dict(color='red', width=2, dash='dash')
))

# Adiciona títulos e rótulos
fig.update_layout(
    title="Gráfico de Dispersão dos Retornos da Estratégia vs. Retornos do S&P 500 Index",
    xaxis_title="Retornos do S&P 500 Index",
    yaxis_title="Retornos da Estratégia"
)

# Exibe o gráfico
fig.show()

##### **Gráfico de barras dos Retornos da Estratégia vs. Retornos do S&P 500 Index**

In [27]:
import plotly.graph_objects as go

# Dados de exemplo
x_values = average_monthly_results.index
y_values_1 = average_monthly_results
y_values_2 = sp500_index_period_returns

# Criação das barras empilhadas
fig = go.Figure()

# Primeira barra (base)
fig.add_trace(go.Bar(
    x=x_values,
    y=y_values_1,
    name=average_monthly_results.name
))

# Segunda barra (sobreposta na anterior)
fig.add_trace(go.Bar(
    x=x_values,
    y=y_values_2,
    name=sp500_index_period_returns.name
))

# Configuração do layout para barras empilhadas
fig.update_layout(
    barmode='stack',
    title='Retornos da Estratégia vs. Retornos do S&P 500 Index',
    xaxis_title='Períodos',
    yaxis_title='Retornos'
)

# Exibe o gráfico
fig.show()


##### **

In [46]:
from scipy import stats

# Suponha que 'average_monthly_results' seja o seu vetor de retornos da estratégia
returns = average_monthly_results  # Substitua com os seus dados de retornos

# Ajusta a distribuição normal aos dados (média e desvio padrão)
mean, std_dev = stats.norm.fit(returns)

# Cria o histograma
hist_data = go.Histogram(
    x=returns,
    nbinsx=30,  # Defina o número de bins conforme necessário
    name='Histograma dos Retornos',
    opacity=0.75,
    histnorm='probability'  # Normaliza o histograma para a probabilidade
)

# Cria a curva normal ajustada
x_range = np.linspace(min(returns), max(returns), 1000)
y_range = stats.norm.pdf(x_range, mean, std_dev)  # Calcula a função de densidade da normal

# Adiciona a curva normal ao gráfico
normal_curve = go.Scatter(
    x=x_range,
    y=y_range,
    mode='lines',
    name='Curva Normal Ajustada',
    line=dict(color='red', width=2)
)

# Calcular a assimetria e a curtose
skewness = stats.skew(returns)
kurtosis = stats.kurtosis(returns)

# Cria a figura com histograma e curva normal
fig = go.Figure(data=[hist_data, normal_curve])

# Adicionar anotações de assimetria e curtose
fig.add_annotation(
    x=0.95, y=0.95, 
    text=f'Assimetria: {skewness:.2f}<br>Curtose: {kurtosis:.2f}', 
    showarrow=False, 
    font=dict(size=12, color="black"), 
    align="center", 
    xref="paper", yref="paper", 
    bgcolor="white", opacity=0.7
)

# Atualiza o layout com títulos e rótulos
fig.update_layout(
    title="Histograma dos Retornos da Estratégia com Curva Normal Ajustada",
    xaxis_title="Retornos",
    yaxis_title="Frequência (Probabilidade)",
    bargap=0.1  # Ajusta o espaçamento entre as barras
)

# Exibe o gráfico
fig.show()


*Assimetria -> quanto mais próximo de 0, mais "normal" é a distribuição*.

*Kurtosis -> quanto mais próximo de 3, mais "normal" é a distribuição*.

# Código do backtest

PRIMEIRO TESTE PARA O BACKTEST
Cálculo do S&P ta errado

In [28]:
# Valor inicial do portfólio
valor_portfolio_inicial = 100000

# Listas para armazenar o histórico do portfólio e as datas correspondentes
historico_portfolio = []
datas_portfolio = []

# Ler todas as planilhas do arquivo Excel
sheets = pd.read_excel('resultados_portfolio_mensal_full_modified.xlsx', sheet_name=None, header=0)

for nome_sheet, df in sheets.items():
    # Verificar se o DataFrame está vazio
    if df.empty:
        print(f"A planilha '{nome_sheet}' está vazia. Pulando este período.")
        continue

    # Verificar se as colunas esperadas estão presentes
    colunas_esperadas = ['Acao', 'Retorno Predito', 'Retorno Real']
    if not all(col in df.columns for col in colunas_esperadas):
        print(f"A planilha '{nome_sheet}' não contém as colunas esperadas. Pulando este período.")
        continue

    # Extrair o ticker, data de início e data de fim da coluna 'Acao'
    try:
        # Aplicar a extração em cada linha
        def extrair_dados(row):
            acao_periodo = row['Acao']
            if pd.isnull(acao_periodo):
                return pd.Series([np.nan, np.nan, np.nan])
            parts = acao_periodo.split(':')
            simbolo = parts[0].strip()
            datas = parts[1].strip().split(' - ')
            data_inicio = pd.to_datetime(datas[0].strip(), format='%Y-%m-%d')
            data_fim = pd.to_datetime(datas[1].strip(), format='%Y-%m-%d')
            return pd.Series([simbolo, data_inicio, data_fim])

        df[['Simbolo', 'Data_Inicio', 'Data_Fim']] = df.apply(extrair_dados, axis=1)
        # Remover linhas com dados ausentes
        df.dropna(subset=['Simbolo', 'Data_Inicio', 'Data_Fim'], inplace=True)
    except Exception as e:
        print(f"Erro ao extrair os dados da planilha '{nome_sheet}': {e}")
        continue

    # Obter a data de início e fim do período a partir da primeira linha
    data_inicio = df['Data_Inicio'].iloc[0]
    data_fim = df['Data_Fim'].iloc[0]

    # Ordenar pelo retorno predito
    df = df.sort_values(by='Retorno Predito', ascending=False)

    # Selecionar top 10%
    n_acoes = max(int(len(df) * 0.10), 1)
    acoes_compradas = df.head(n_acoes)

    # Alocar capital igualmente entre as ações
    valor_portfolio = valor_portfolio_inicial if not historico_portfolio else historico_portfolio[-1]
    alocacao_por_acao = valor_portfolio / n_acoes

    # Calcular o retorno real
    retornos = acoes_compradas['Retorno Real'] / 100  # Converter porcentagem para decimal
    valor_final = alocacao_por_acao * (1 + retornos)

    # Atualizar o valor do portfólio
    valor_portfolio = valor_final.sum()
    historico_portfolio.append(valor_portfolio)
    datas_portfolio.append(data_fim)

    # Imprimir os resultados do período
    print(f"Período: {data_inicio.date()} - {data_fim.date()}")
    print(f"Ações compradas: {acoes_compradas['Simbolo'].tolist()}")
    print("Retorno de cada ação:")
    for index, row in acoes_compradas.iterrows():
        print(f" - {row['Simbolo']}: {row['Retorno Real']}%")
    print(f"Valor do portfólio ao final do período: ${valor_portfolio:.2f}\n")

# Verificar se o histórico do portfólio está vazio
if not historico_portfolio or not datas_portfolio:
    print("Nenhum período foi processado com sucesso. Verifique os dados e tente novamente.")
else:
    # Criar DataFrame com o histórico do portfólio
    df_portfolio = pd.DataFrame({
        'Data': datas_portfolio,
        'Valor': historico_portfolio
    })
    # Ordenar por data
    df_portfolio.sort_values(by='Data', inplace=True)
    df_portfolio.reset_index(drop=True, inplace=True)

    # 5. Obter dados do S&P500
    data_inicio_total = df_portfolio['Data'].iloc[0] - timedelta(days=5)
    data_fim_total = df_portfolio['Data'].iloc[-1] + timedelta(days=5)
    sp500 = yf.download('^GSPC', start=data_inicio_total, end=data_fim_total)
    print("Colunas disponíveis em sp500:", sp500.columns)

    # Determinar a coluna de preço a ser usada
    if 'Adj Close' in sp500.columns:
        price_column = 'Adj Close'
    elif 'Close' in sp500.columns:
        price_column = 'Close'
    else:
        raise KeyError("Nenhuma coluna de preço ('Adj Close' ou 'Close') encontrada em sp500")

    # Resetar o índice para ter a coluna 'Date'
    sp500.reset_index(inplace=True)

    # Remover informações de fuso horário da coluna 'Date'
    sp500['Date'] = sp500['Date'].dt.tz_localize(None)

    # Verificar os tipos de dados das colunas de datas
    print("dtype de sp500['Date']:", sp500['Date'].dtype)
    print("dtype de df_portfolio['Data']:", df_portfolio['Data'].dtype)

    # Calcular retorno acumulado do S&P500
    sp500['Retorno'] = sp500[price_column].pct_change().fillna(0)
    sp500['Retorno Acumulado'] = (1 + sp500['Retorno']).cumprod()

    # Interpolar o retorno acumulado do S&P500 nas datas do portfólio
    sp500_interp = sp500[['Date', 'Retorno Acumulado']].set_index('Date').reindex(
        df_portfolio['Data'], method='ffill')

    # Após reindexar, resetamos o índice
    sp500_interp.reset_index(inplace=True)

    # Verificar as colunas e as primeiras linhas
    print("Colunas de sp500_interp após reset_index:", sp500_interp.columns)
    print("Primeiras linhas de sp500_interp:")
    print(sp500_interp.head())

    # Ajustar o acesso à coluna de datas de sp500_interp
    if 'Date' in sp500_interp.columns:
        date_column = 'Date'
    elif 'index' in sp500_interp.columns:
        date_column = 'index'
    elif 'Data' in sp500_interp.columns:
        date_column = 'Data'
    else:
        print("A coluna de datas não foi encontrada em sp500_interp.")
        print("Colunas disponíveis:", sp500_interp.columns)
        raise KeyError("Coluna de datas não encontrada em sp500_interp")

    # Calcular retorno acumulado da estratégia
    df_portfolio['Retorno'] = df_portfolio['Valor'].pct_change().fillna(0)
    df_portfolio['Retorno Acumulado'] = (1 + df_portfolio['Retorno']).cumprod()

    # 6. Plotar o gráfico comparativo usando Plotly
    trace_estrategia = go.Scatter(
        x=df_portfolio['Data'],
        y=df_portfolio['Retorno Acumulado'],
        mode='lines+markers',
        name='Retorno Acumulado da Estratégia',
        yaxis='y1'
    )

    trace_sp500 = go.Scatter(
        x=sp500_interp[date_column],
        y=sp500_interp['Retorno Acumulado'],
        mode='lines+markers',
        name='Retorno Acumulado do S&P500',
        yaxis='y1'
    )

    trace_capital = go.Scatter(
        x=df_portfolio['Data'],
        y=df_portfolio['Valor'],
        mode='lines+markers',
        name='Capital do Portfólio',
        yaxis='y2'
    )

    data = [trace_estrategia, trace_sp500, trace_capital]

    layout = go.Layout(
        title='Comparação de Retorno Acumulado e Capital do Portfólio',
        xaxis=dict(title='Data'),
        yaxis=dict(
            title='Retorno Acumulado',
            side='left',
            showgrid=False,
            zeroline=False
        ),
        yaxis2=dict(
            title='Capital do Portfólio',
            overlaying='y',
            side='right',
            showgrid=False,
            zeroline=False
        ),
        legend=dict(x=0.01, y=0.99)
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()

# 7. Plotar o histograma da distribuição dos retornos da estratégia

# Remover possíveis NaNs ou valores infinitos
retornos = df_portfolio['Retorno'].dropna().replace([np.inf, -np.inf], np.nan).dropna()

# Criar o histograma dos retornos
histograma = go.Figure(data=[go.Histogram(
    x=retornos * 100,  # Converter para porcentagem
    xbins=dict(
        start=retornos.min() * 100,
        end=retornos.max() * 100,
        size=1  # Intervalo de 1% entre as barras
    ),
    marker_color='blue',
    opacity=0.75
)])

histograma.update_layout(
    title='Distribuição dos Retornos da Estratégia',
    xaxis_title='Retorno (%)',
    yaxis_title='Frequência',
    bargap=0.2,
    bargroupgap=0.1
)

FileNotFoundError: [Errno 2] No such file or directory: 'resultados_portfolio_mensal_full_modified.xlsx'

TESTE 2 pq o primeiro tava dando problema com o S&P500

Tudo fragmentado pra facilitar identificação de erro

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import timedelta
import plotly.graph_objs as go

# Valor inicial do portfólio
valor_portfolio_inicial = 100000

# Listas para armazenar o histórico do portfólio e as datas correspondentes
historico_portfolio = []
datas_portfolio = []
datas_inicio_portfolio = []
retornos_estrategia_periodo = []
retornos_sp500_periodo = []

# Nome do arquivo Excel
arquivo_excel = 'resultados_portfolio_mensal_full_modified.xlsx'

# Tentar ler o arquivo Excel
try:
    sheets = pd.read_excel(arquivo_excel, sheet_name=None, header=0)
    print(f"Arquivo '{arquivo_excel}' lido com sucesso.")
except FileNotFoundError:
    print(f"Arquivo '{arquivo_excel}' não encontrado. Verifique o caminho e o nome do arquivo.")
    exit()

# Função para extrair dados de cada linha
def extrair_dados(row):
    acao_periodo = row['Acao']
    if pd.isnull(acao_periodo):
        return pd.Series([np.nan, np.nan, np.nan])
    parts = acao_periodo.split(':')
    if len(parts) != 2:
        return pd.Series([np.nan, np.nan, np.nan])
    simbolo = parts[0].strip()
    datas = parts[1].strip().split(' - ')
    if len(datas) != 2:
        return pd.Series([np.nan, np.nan, np.nan])
    try:
        data_inicio = pd.to_datetime(datas[0].strip(), dayfirst=True).normalize()
        data_fim = pd.to_datetime(datas[1].strip(), dayfirst=True).normalize()
    except ValueError:
        return pd.Series([np.nan, np.nan, np.nan])
    return pd.Series([simbolo, data_inicio, data_fim])

# Processar cada planilha
for nome_sheet, df in sheets.items():
    print(f"\nProcessando a planilha: {nome_sheet}")
    
    # Verificar se o DataFrame está vazio
    if df.empty:
        print(f"A planilha '{nome_sheet}' está vazia. Pulando este período.")
        continue

    # Verificar se as colunas esperadas estão presentes
    colunas_esperadas = ['Acao', 'Retorno Predito', 'Retorno Real']
    if not all(col in df.columns for col in colunas_esperadas):
        print(f"A planilha '{nome_sheet}' não contém as colunas esperadas: {colunas_esperadas}. Pulando este período.")
        continue

    # Aplicar a extração em cada linha
    df[['Simbolo', 'Data_Inicio', 'Data_Fim']] = df.apply(extrair_dados, axis=1)

    # Remover linhas com dados ausentes
    linhas_antes = len(df)
    df.dropna(subset=['Simbolo', 'Data_Inicio', 'Data_Fim'], inplace=True)
    linhas_depois = len(df)
    print(f"Linhas processadas: {linhas_depois} (removidas {linhas_antes - linhas_depois} linhas com dados ausentes)")

    # Verificar se após a extração o DataFrame está vazio
    if df.empty:
        print(f"A planilha '{nome_sheet}' não possui dados válidos após a extração. Pulando este período.")
        continue

    # Obter a data de início e fim do período a partir da primeira linha
    data_inicio = df['Data_Inicio'].iloc[0]
    data_fim = df['Data_Fim'].iloc[0]

    print(f"Período: {data_inicio.date()} - {data_fim.date()}")

    # Ordenar pelo retorno predito
    df = df.sort_values(by='Retorno Predito', ascending=False)

    # Selecionar top 10%
    n_acoes = max(int(len(df) * 0.10), 1)
    acoes_compradas = df.head(n_acoes)
    print(f"Selecionadas {n_acoes} ações para o portfólio.")

    # Alocar capital igualmente entre as ações
    if not historico_portfolio:
        valor_portfolio_anterior = valor_portfolio_inicial
    else:
        valor_portfolio_anterior = historico_portfolio[-1]
    valor_portfolio = valor_portfolio_anterior
    alocacao_por_acao = valor_portfolio / n_acoes

    # Calcular o retorno real
    retornos = acoes_compradas['Retorno Real'] / 100  # Converter porcentagem para decimal
    valor_final = alocacao_por_acao * (1 + retornos)

    # Atualizar o valor do portfólio
    valor_portfolio = valor_final.sum()
    historico_portfolio.append(valor_portfolio)
    datas_portfolio.append(data_fim)
    datas_inicio_portfolio.append(data_inicio)

    # Calcular o retorno do período
    retorno_periodo = (valor_portfolio - valor_portfolio_anterior) / valor_portfolio_anterior * 100  # Em porcentagem
    retornos_estrategia_periodo.append(retorno_periodo)

    # Imprimir os resultados do período
    print(f"Ações compradas: {acoes_compradas['Simbolo'].tolist()}")
    print("Retorno de cada ação:")
    for index, row in acoes_compradas.iterrows():
        print(f" - {row['Simbolo']}: {row['Retorno Real']}%")
    print(f"Retorno do período: {retorno_periodo:.2f}%")
    print(f"Valor do portfólio ao final do período: ${valor_portfolio:.2f}\n")


In [ ]:
# Verificar se o histórico do portfólio está vazio
if not historico_portfolio or not datas_portfolio:
    print("Nenhum período foi processado com sucesso. Verifique os dados e tente novamente.")
    exit()
else:
    # Criar DataFrame com o histórico do portfólio
    df_portfolio = pd.DataFrame({
        'Data': datas_portfolio,
        'Valor': historico_portfolio
    })
    # Ordenar por data
    df_portfolio.sort_values(by='Data', inplace=True)
    df_portfolio.reset_index(drop=True, inplace=True)

    print("\nHistórico do Portfólio:")
    print(df_portfolio)


In [ ]:
# Obter dados do S&P500
data_inicio_total = min(datas_inicio_portfolio) - timedelta(days=5)
data_fim_total = max(datas_portfolio) + timedelta(days=5)
print(f"\nBaixando dados do S&P500 de {data_inicio_total.date()} a {data_fim_total.date()}.")
    
try:
    # Baixar dados do S&P500 sem o parâmetro 'group_by'
    sp500 = yf.download('^GSPC', start=data_inicio_total, end=data_fim_total)
    print("Dados do S&P500 baixados com sucesso.")
except Exception as e:
    print(f"Erro ao baixar dados do S&P500: {e}")
    exit()

# Resetar o índice para transformar o índice em coluna
sp500.reset_index(inplace=True)

# Verificar se as colunas são MultiIndex
if isinstance(sp500.columns, pd.MultiIndex):
    # Achatar as colunas combinando os níveis
    sp500.columns = [' '.join(col).strip() for col in sp500.columns.values]
    print("Ajustando colunas de MultiIndex para nível único.")
else:
    print("Colunas de sp500 são de nível único.")

# Converter os nomes das colunas para minúsculas para facilitar a comparação
sp500.columns = [col.lower() for col in sp500.columns]
print(f"Colunas em sp500 após conversão para minúsculas: {sp500.columns.tolist()}")

# Verificar o nome da coluna de data
date_column_name = None
for col in sp500.columns:
    if 'date' in col:
        date_column_name = col
        break

if date_column_name is None:
    print("Nenhuma coluna de data encontrada em sp500.")
    exit()

if date_column_name != 'date':
    sp500.rename(columns={date_column_name: 'date'}, inplace=True)
    print(f"Renomeada a coluna de data para 'date'.")

# Remover informações de fuso horário e normalizar a data
sp500['date'] = pd.to_datetime(sp500['date']).dt.tz_localize(None).dt.normalize()

# Determinar a coluna de preço a ser usada
price_column = None
for col in sp500.columns:
    if 'adj close' in col.lower():
        price_column = col
        break
if price_column is None:
    for col in sp500.columns:
        if 'close' in col.lower():
            price_column = col
            break

if price_column is None:
    raise KeyError(f"Nenhuma coluna de preço ('Adj Close' ou 'Close') encontrada em sp500. Colunas disponíveis: {sp500.columns.tolist()}")

print(f"Coluna de preço selecionada: '{price_column}'")

# Criar sp500_prices com 'date' como coluna
sp500_prices = sp500[['date', price_column]].rename(columns={price_column: 'Price'})

print(f"Colunas em sp500_prices: {sp500_prices.columns.tolist()}")

# Calcular Retorno do S&P500 (não acumulado)
sp500_prices['Retorno'] = sp500_prices['Price'].pct_change().fillna(0)

# Verificar se 'Retorno' foi criado
if 'Retorno' not in sp500_prices.columns:
    print("Erro: A coluna 'Retorno' não foi criada em sp500_prices.")
    print(f"Colunas disponíveis em sp500_prices: {sp500_prices.columns.tolist()}")
    exit()
else:
    print("'Retorno' criado com sucesso em sp500_prices.")
    print(sp500_prices[['date', 'Price', 'Retorno']].head())


In [ ]:
# Preparar required_dates
required_dates = pd.Series(datas_inicio_portfolio + datas_portfolio).drop_duplicates()
required_dates = pd.to_datetime(required_dates).dt.normalize()
required_dates = required_dates.to_frame(name='date')
required_dates.sort_values('date', inplace=True)
required_dates.reset_index(drop=True, inplace=True)

# Garantir que não há duplicatas em required_dates
required_dates.drop_duplicates(subset='date', inplace=True)

# Realizar o merge_asof para alinhar os preços do S&P500 com as datas do portfólio
prices_at_required_dates = pd.merge_asof(
    required_dates.sort_values('date'),
    sp500_prices.sort_values('date'),
    on='date',
    direction='backward'
)

# Verificar se há NaNs após o merge
na_prices = prices_at_required_dates['Price'].isna().sum()
if na_prices > 0:
    print(f"Atenção: {na_prices} datas no portfólio não têm correspondência no S&P500.")

# Criar um dicionário mapeando datas a preços
date_price_dict = dict(zip(prices_at_required_dates['date'], prices_at_required_dates['Price']))


In [ ]:
# Calcular os retornos do S&P500 para cada período
for i in range(len(datas_inicio_portfolio)):
    data_inicio = datas_inicio_portfolio[i].normalize()
    data_fim = datas_portfolio[i].normalize()

    price_inicio = date_price_dict.get(data_inicio)
    price_fim = date_price_dict.get(data_fim)

    if pd.isna(price_inicio) or pd.isna(price_fim):
        print(f"Não foi possível obter preços para o período {data_inicio.date()} - {data_fim.date()}")
        retorno_sp500_periodo = np.nan
    else:
        retorno_sp500_periodo = (price_fim - price_inicio) / price_inicio * 100
        print(f"Retorno do S&P500 no período {data_inicio.date()} - {data_fim.date()}: {retorno_sp500_periodo:.2f}%")
    retornos_sp500_periodo.append(retorno_sp500_periodo)


In [ ]:
# Criar DataFrame com os retornos por período
df_returns = pd.DataFrame({
    'Data_Inicio': datas_inicio_portfolio,
    'Data_Fim': datas_portfolio,
    'Retorno_Estrategia': retornos_estrategia_periodo,
    'Retorno_SP500': retornos_sp500_periodo
})

# Remover períodos onde o retorno do S&P500 é NaN
df_returns.dropna(subset=['Retorno_SP500'], inplace=True)

# Formatar as datas para serem usadas como rótulos no gráfico
df_returns['Período'] = df_returns['Data_Fim'].dt.strftime('%Y-%m-%d')

print("\nDataFrame com Retornos por Período:")
print(df_returns)


In [ ]:
# Plotar o gráfico de barras duplo dos retornos por período
fig_bar = go.Figure(data=[
    go.Bar(name='Estratégia', x=df_returns['Período'], y=df_returns['Retorno_Estrategia']),
    go.Bar(name='S&P500', x=df_returns['Período'], y=df_returns['Retorno_SP500'])
])

fig_bar.update_layout(
    barmode='group',
    title='Retornos por Período: Estratégia vs S&P500',
    xaxis_title='Período',
    yaxis_title='Retorno (%)',
    legend=dict(x=0.01, y=0.99)
)

fig_bar.show()


AJEITAR O GRÁFICO QUE FICOU RUIM(provavelmente por conta da escala)

# **Sugestões**

**Gŕaficos a serem feitos:** *(Ver em quais gráficos deve ser plotada uma região de confiança)*

    - Gráfico de série temporal com o maior retorno acumulado, menor retorno acumulado e retorno médio acumulado da estratégia.
  
    - Gráfico de série temporal com o retorno médio por período da estratégia x sharpe ratio por período da estratégia x sortino ratio por  período da estratégia.